In [239]:
import openai
import os
import load_config

KEY = os.getenv("KEY")
openai.api_key = KEY

In [22]:
import json
from pprint import pprint

debates = [json.loads(_) for _ in open("data/pmqs_parsed_2022-11-30.jsonl", "r")]

questions = []
for _ in debates:
    question = _["question"]
    if "member" in question:
        questions.extend(question["member"])

    else:
        questions.extend(question["opposition"])

sample = questions[1:11]

In [25]:
# TODO: Sample of 10
# TODO: Argument Map

prompt = "Can you identify the claims and premises in the following text: "
prompt + sample[1]

for _ in sample:
    print(prompt + _)

Can you identify the claims and premises in the following text: In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?
Can you identify the claims and premises in the following text: I have just returned from the south Pacific and—[Interruption.] I have resisted the All Blacks jokes. I found a deep concern there about the expanding tentacles of communist China. Does my right hon. Friend agree that China is more than just “a systemic challenge”, as he has put it, and is in fact an expanding, serious geopolitical threat? (902524)
Can you identify the claims and premises in the following text: I join the Prime Minister in saying,

In [28]:
def gpt_batch(sample):
    prompt = "Identify the claims and premises in the following text: "

    parsed = []
    for _ in sample:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt= prompt  + _,
            temperature=1,
            max_tokens=2500
        )
        parsed.append(response)

    return parsed

In [31]:
result = gpt_batch(sample)

In [32]:
result

[<OpenAIObject text_completion id=cmpl-6TEJge3jfTGkVNVnIMDSHH8NPrhAr at 0x1302ba860> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nClaim: The Prime Minister should bring forward legislation to allow the people of Scotland to exercise the choice of becoming an independent country. \nPremise 1: David Cameron signed up to the Smith commission. \nPremise 2: The Smith commission promised that nothing in the report would stop Scotland becoming an independent country should the people choose. \nPremise 3: The Supreme Court judgment has passed."
     }
   ],
   "created": 1672424564,
   "id": "cmpl-6TEJge3jfTGkVNVnIMDSHH8NPrhAr",
   "model": "text-davinci-003",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 82,
     "prompt_tokens": 97,
     "total_tokens": 179
   }
 },
 <OpenAIObject text_completion id=cmpl-6TEJjUz3muD2TBfqDtxLxNhM0bwDX at 0x1302a1400> JSON: {
   "choices": [
     {
     

In [46]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joshua_sheppard/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [219]:
import re
from nltk.tokenize import sent_tokenize

parsed_args = []
counta = 0

for _ in result:
    arg = {}

    claim_premises = _["choices"][0]["text"].split("\n")
    claim_premises = list(filter(lambda x: (x.lower().strip() != ""), claim_premises))

    claim_pattern = re.compile("Claim*[\s\d]*:")
    premise_pattern = re.compile("Premise*[\s\d]*:")

    arg["id_"] = counta
    arg["debate"] = sample[counta]
    arg["claims"] = [_.strip() for _ in filter(claim_pattern.match, claim_premises)]
    arg["premises"] = [_.strip() for _ in filter(premise_pattern.match, claim_premises)]

    parsed_args.append(arg)
    counta += 1

parsed_args

[{'id_': 0,
  'debate': 'In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?',
  'claims': ['Claim: The Prime Minister should bring forward legislation to allow the people of Scotland to exercise the choice of becoming an independent country.'],
  'premises': ['Premise 1: David Cameron signed up to the Smith commission.',
   'Premise 2: The Smith commission promised that nothing in the report would stop Scotland becoming an independent country should the people choose.',
   'Premise 3: The Supreme Court judgment has passed.']},
 {'id_': 1,
  'debate': 'I have just returned from the south Pacific and—[Interruption.] I have r

In [118]:
debate = parsed_args[1]
claim = re.sub(claim_pattern, "", debate["claims"][0]).strip()
premise_1 = re.sub(premise_pattern, "", debate["premises"][0]).strip()

'I have just returned from the south Pacific and found a deep concern there about the expanding tentacles of communist China.'

In [229]:
# TODO: Graphviz
# TODO: Dash
# TODO: Entity Recognition
# TODO: Entity Linking

In [221]:
parsed_args

[{'id_': 0,
  'debate': 'In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?',
  'claims': ['Claim: The Prime Minister should bring forward legislation to allow the people of Scotland to exercise the choice of becoming an independent country.'],
  'premises': ['Premise 1: David Cameron signed up to the Smith commission.',
   'Premise 2: The Smith commission promised that nothing in the report would stop Scotland becoming an independent country should the people choose.',
   'Premise 3: The Supreme Court judgment has passed.']},
 {'id_': 1,
  'debate': 'I have just returned from the south Pacific and—[Interruption.] I have r

In [238]:
import graphviz

for parsed in parsed_args:
    dot = graphviz.Digraph(filename="pmqs")

    claim = parsed["claims"]
    premises = parsed["premises"]

    """ a -> b -> c """

    adus = claim + premises
    idx = list(range(len(claim + premises)))

    for i, _ in zip(idx, adus):
        dot.node(str(i), _)
        if i < len(idx) - 1:
            dot.edge(str(i), str(i + 1))


    pmq = parsed["id_"]
    dot.render(f"./data/arg_maps/pmqs_{pmq}.gv")

    print(dot.source)

digraph {
	0 [label="Claim: The Prime Minister should bring forward legislation to allow the people of Scotland to exercise the choice of becoming an independent country."]
	0 -> 1
	1 [label="Premise 1: David Cameron signed up to the Smith commission."]
	1 -> 2
	2 [label="Premise 2: The Smith commission promised that nothing in the report would stop Scotland becoming an independent country should the people choose."]
	2 -> 3
	3 [label="Premise 3: The Supreme Court judgment has passed."]
}

digraph {
	0 [label="Claim: China is more than just \"a systemic challenge\" and is in fact an expanding, serious geopolitical threat"]
	0 -> 1
	1 [label="Premise: I have just returned from the south Pacific and found a deep concern there about the expanding tentacles of communist China."]
}

digraph {
	0 [label="Claim: Why did the Prime Minister hand Winchester nearly £6 million of taxpayers' money this year?"]
	0 -> 1
	1 [label="Premise 1: Winchester College charges more than £45,000 a year in fees

In [184]:
# from pyvis.network import Network
# from IPython.core.display import display, HTML
#
# net = Network(notebook = True)
#
# nodes = ["claim", "premise_1"]
# titles = [claim, premise_1]
# net.add_node(claim, title="claim")
# net.add_node(premise_1, title="premise")
# net.add_edge(premise_1, claim)
#
# net.show("test.html")
# display(HTML("test.html"))

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


/var/folders/9y/jc1z3n7n78ndcq81djws05m00000gn/T/ipykernel_44643/2270067679.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
